get mass

In [25]:
%run -i 'get_mass.py'

# Get data from uniprot api

In [1]:
import retrieve_from_xml_uniprotapi as uniprotxml

In [2]:
datos = uniprotxml.get_uniprot_data('P31749')

/homes/dtg30/Desktop/group_proj/venv/src/Group-project/python_scripts/retrieve_from_xml_uniprotapi.py:17: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 17 of the file /homes/dtg30/Desktop/group_proj/venv/src/Group-project/python_scripts/retrieve_from_xml_uniprotapi.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  bs_data = BeautifulSoup(requested_xml.content)


In [3]:
uniprotxml.get_accession(datos)

'P31749'

In [26]:
import sys
del sys.modules["uniprotxml"]
del uniprotxml


KeyError: 'uniprotxml'

# Random things, tests

In [3]:
def get_uniprot_data(accession):
    ''' get data from uniprot api. The data is in xml. This uses bs4 BeautifulSoup to parse it
     for eg: https://www.ebi.ac.uk/proteins/api/coordinates?offset=0&size=100&accession=P31749'''
    text = 'https://www.ebi.ac.uk/proteins/api/coordinates?offset=0&size=100&accession={}'.format(accession)
    try:
        import requests
    except:
        'requests package not installed. Run "pip install -r requirements"'
    try:
        from bs4 import BeautifulSoup
    except:
        print(' bs4 package not installed. Run "pip install -r requirements"')
    try:
        requested_xml = requests.get(text)
    except:
        print('Problem with the following accession number: ' + accession)
    bs_data = BeautifulSoup(requested_xml.content)
    return bs_data

In [4]:
uniprot_id = 'P31749'
datos = get_uniprot_data(uniprot_id)

In [18]:
datos.gnentries.gnentry.protein.findAll('alternativename')[1]


<alternativename><fullname>Proto-oncogene c-Akt</fullname></alternativename>

In [11]:
def get_alternative_prot_names(bs_data):
    ''' get alternative prot name from bs_data from get_uniprot_data,
        returns a list'''
    uniprot_id = bs_data.gnentries.gnentry.accession.text
    # get list of all alternative names
    all_alternatives = bs_data.gnentries.gnentry.protein.findAll('alternativename')
    tmp_alternatives = [] # null list
    # append to list
    for alternative in all_alternatives:
        text_list = [uniprot_id, alternative.text]
        text_output = '|'.join(text_list)
        tmp_alternatives.append(text_output)
    return tmp_alternatives

get_alternative_prot_names(datos)

['P31749|RAC-PK-alpha',
 'P31749|Proto-oncogene c-Akt',
 'P31749|Protein kinase B alphaPKB alpha',
 'P31749|Protein kinase BPKB']

In [4]:
def data_modif_res(bs_data):
    ''' returns a xml list of modif residues, with all information'''
    reference = bs_data.gnentries.gnentry.findAll('gncoordinate')[0]
    features = reference.findAll('feature')
    phospho_list = []
    for entry in features:
        if entry['type'] == 'modified residue': # extract modified residues
             if 'hospho' in entry.findAll('ns2:description')[0].text: # is phosphosomething ?
                 phospho_list.append(entry)
    return phospho_list

In [38]:
# modif residues: position, type, genom_begin, genom_end
def modres_gral_info(bs_data, uniprot_id):
    mod_res_list = data_modif_res(bs_data)
    tmp_list = []
    
    for mod_res_item in mod_res_list:
        
        residue_position = str(mod_res_item.findAll('ns2:position')[0]['position']) # position
        type_modif = mod_res_item.findAll('ns2:description')[0].text # type of modif
        
        genom_begin = str(mod_res_item.findAll('ns2:begin')[0]['position']) #genom_beg_pos
        genom_end = str(mod_res_item.findAll('ns2:end')[0]['position']) #genom_end_pos
        
        text_list = [uniprot_id, residue_position, type_modif, genom_begin, genom_end]
        text_row = '|'.join(text_list)
        
        tmp_list.append(text_row)
        
    return tmp_list

mod_resid_results = modres_gral_info(datos, uniprot_id)
print(mod_resid_results)

['P31749|450|Phosphothreonine.|104770760|104770758', 'P31749|176|Phosphotyrosine; by TNK2.|104775117|104775115', 'P31749|473|Phosphoserine; by IKKE, MTOR and TBK1; alternate.|104770367|104770365', 'P31749|129|Phosphoserine; alternate.|104775702|104775700', 'P31749|308|Phosphothreonine; by IKKE, PDPK1 and TBK1.|104773286|104773284', 'P31749|448|Phosphothreonine.|104770766|104770764', 'P31749|126|Phosphoserine; alternate.|104775711|104775709', 'P31749|124|Phosphoserine.|104775717|104775715', 'P31749|474|Phosphotyrosine.|104770364|104770362']


In [39]:
# references: uniprot_id, position, references[0], references[1]

def modres_references(bs_data, uniprot_id):
    mod_res_list = data_modif_res(bs_data)
    tmp_main_list = []
    
    for mod_res_item in mod_res_list:
        residue_position = mod_res_item.findAll('ns2:position')[0]['position'] # position
        
        references = mod_res_item.findAll('ns2:dbreference') # position of modif residue
        tmp_ref_list = []
        for reference in references:
            tmp_tup = (reference['id'],reference['type']) # (ref_id,ref_type)
            tmp_ref_list.append(tmp_tup)
        
        for item in tmp_ref_list:
            text_list = [uniprot_id, residue_position, item[0], item[1]]
            text_output = '|'.join(text_list)
            tmp_main_list.append(text_output)
    
    return tmp_main_list
    
tmp = modres_references(datos, uniprot_id)
print(tmp)

['P31749|450|24275569|PubMed', 'P31749|176|20333297|PubMed', 'P31749|473|9736715|PubMed', 'P31749|473|8978681|PubMed', 'P31749|473|23799035|PubMed', 'P31749|473|15047712|PubMed', 'P31749|473|21464307|PubMed', 'P31749|473|16266983|PubMed', 'P31749|473|15718470|PubMed', 'P31749|473|20333297|PubMed', 'P31749|473|17013611|PubMed', 'P31749|473|14761976|PubMed', 'P31749|473|20978158|PubMed', 'P31749|129|23186163|PubMed', 'P31749|129|18669648|PubMed', 'P31749|129|17081983|PubMed', 'P31749|308|15718470|PubMed', 'P31749|308|9512493|PubMed', 'P31749|308|18456494|PubMed', 'P31749|308|8978681|PubMed', 'P31749|308|20333297|PubMed', 'P31749|308|21464307|PubMed', 'P31749|308|20481595|PubMed', 'P31749|448|24275569|PubMed', 'P31749|126|18669648|PubMed', 'P31749|124|18669648|PubMed', 'P31749|474|12149249|PubMed']


In [52]:
def get_ensembl_geneid(bs_data):
    ''' get ensembl gene id name from bs_data from get_uniprot_data,
        returns a list'''
    genes_ids = []
    genes = bs_data.gnentries.gnentry.findAll('gncoordinate')
    for gene in genes:
        gene_ids.append(gene['ensembl_gene_id'] )
    return genes_ids
    # should return a list
get_ensembl_geneid(datos)

NameError: name 'gene_ids' is not defined

In [46]:
def get_alternative_prot_names(bs_data):
    ''' get alternative prot name from bs_data from get_uniprot_data,
        returns a list'''
    uniprot_id = bs_data.gnentries.gnentry.accession.text
    # get list of all alternative names
    all_alternatives = bs_data.gnentries.gnentry.protein.findAll('alternativename')
    tmp_alternatives = [] # null list
    # append to list
    for alternative in all_alternatives:
        text_list = [uniprot_id, alternative.text]
        text_output = '|'.join(text_list)
        tmp_alternatives.append(text_output)
    return tmp_alternatives
    # should return a list

get_alternative_prot_names(datos)    

['P31749|RAC-PK-alpha',
 'P31749|Proto-oncogene c-Akt',
 'P31749|Protein kinase B alphaPKB alpha',
 'P31749|Protein kinase BPKB']

# RANDOM

In [29]:
from bs4 import BeautifulSoup

x="""<foo>
   <bar>
      <type foobar="1"/>
      <type foobar="2"/>
   </bar>
</foo>"""

y=BeautifulSoup(x)
print(y.foo.bar)

print(y.foo.bar.findAll("type"))
print(y.foo.bar.findAll("type")[0]["foobar"])

<bar>
<type foobar="1"></type>
<type foobar="2"></type>
</bar>
[<type foobar="1"></type>, <type foobar="2"></type>]
1


In [1]:
from bs4 import BeautifulSoup
import requests

In [34]:
import requests
text = 'https://www.ebi.ac.uk/proteins/api/coordinates?offset=0&size=100&accession=P31749'
aa = requests.get(text)